In [40]:
load("burnsidelib.py")

In [41]:
# Take an element X of a Burnside ring of a finite group G
# and a nonnegative integer k. Computes the k-th Siebeneicher power of X.
# This is (in particular) a G-set whose permutation representation is the k-th
# exterior power of the permutation representation of X.
def siebeneicher_operation(X,k):
    A = X.parent()
    nG = A._num_cc_subgroups # dimension of the table of marks
    repsG = A._cc_reps # reps of conjugacy classes
    marks_in = X.marks()
    marks_out = vector(ZZ,nG)
    R.<t> = PowerSeriesRing(QQ, default_prec=k+1)
    for i in range(nG):
        res_X = X.res(repsG[i])
        matH = res_X.parent().table_of_marks()
        mark_H_lprime_t_X = prod((1-(-t)^(matH[j][0]))^c for j,c in enumerate(res_X.coeffs()))
        marks_out[i] = mark_H_lprime_t_X.padded_list(k+1)[k]
    return A.from_marks(marks_out)

In [110]:
from sage.combinat.combinat import eulerian_polynomial

G = gap("SmallGroup(4,2)") # Klein four group

A_G = BurnsideRing(G)
X = A_G.from_vec([1,0,0,0,0]) # P is a G-invariant polytope in the permutation representation of X
d = X.marks()[0] # d is the dimension of the permutation representation

# a[0], ..., a[d] are the coefficients of the polynomial function t \mapsto #(tP \cap Z^n)
# So we work over the polynomial ring S = A(G)[a_0, ..., a_d]
S = PolynomialRing(A_G, d+1, 'a')
a = S.gens()

R.<t> = PowerSeriesRing(S, 't') # R = A(G)[a_0, ..., a_d][[t]]
# The eulerian polynomials defined by sage are shifted from ours
def f(i):
    return t*eulerian_polynomial(i+1)

# Compute the Equivariant Ehrhart series of P!!
h_star = sum(a[i]*f(i)*(1-t)^(-i) for i in range(d+1))*sum(siebeneicher_operation(X,i)*(-t)^i for i in range(d+1))

# We want the coefficients after t^d to be zero in the representation ring. So, we should compute the kernel of the linearization map.
linearization_map = matrix(ZZ, map(lambda c: c.ValuesOfClassFunction(), G.CharacterTable().PermCharsTom(A_G._tom)))

# M is the matrix such that (a_0 ... a_d) * M = 0 forces the coefficients of t^(d+1), ..., t^(d_max) in h^* to be zero in the representation ring.
d_max = 3*d
h_star_coeffs = h_star.padded_list(d_max+1)
blocks = [matrix(ZZ, (h_star_coeffs[i].monomial_coefficient(a[j]).coeffs() * linearization_map for j in range(d+1))) for i in range(d+1, d_max+1)]
M = block_matrix(blocks, nrows = 1)

# So we need (a_0 ... a_d) to be in the left kernel of M.
M.left_kernel().basis()

[(1, 0, -1, 0, 0), (0, 1, 1, 0, 0)]